### Connecting to the Stat-Xplore API using Python

In [39]:
from statxplore import http_session
from statxplore import objects
import pandas as pd
import numpy as np
import json
import pathlib
import os
import shutil

In [32]:
session = http_session.StatSession(api_key='your_api_key_here')

In [33]:
myPath = (f"C:\\your\\file\\path\\here")
queriedPath = (f"C:\\your\\file\\path\\here\\Queried")

In [46]:
result = []  # Initialize an empty list to store results
file_count = 0
for json_file in pathlib.Path(myPath).glob('*.json'):
    file_name = json_file.name
    file_count = file_count + 1
    
    with open(json_file) as f:  # Use 'with' to handle file opening and closing automatically
        query = json.load(f)
    
    queryString = json.dumps(query)
    data = objects.Table.query_json(session, queryString)  # Initialize session
    
    shutil.move(json_file, queriedPath) #move the queried file to a new directory
    series = pd.Series(data)
    
    # Access row-level information
    rows = [item['labels'][0] for item in series.fields[0]['items']]
    
    # Access column headers
    cols = [item['labels'][0] for item in series.fields[1]['items']]
    
    # Access wafer values
    wafer = [item['labels'][0] for item in series.fields[2]['items']]
    
    # Create new column headers by combining cols and wafer
    newColHeaders = [f"{col} - {waferItem}" for col in cols for waferItem in wafer]
    
    # Retrieve cube data and reshape the values list into a 2D array
    cube_data = series.cubes
    if cube_data:  # Ensure cube_data is not empty
        key = next(iter(cube_data))
        values_list = cube_data[key]['values']
        
        reshaped_values = np.array(values_list).reshape(-1, len(newColHeaders))
        
        # Create DataFrame with reshaped values and correct column headers
        df = pd.DataFrame(reshaped_values, columns=newColHeaders)
        df['category'] = rows
        df['file'] = file_name
        
        # Append the DataFrame to the result list
        result.append(df)

# Now, 'result' contains a list of DataFrames, one for each JSON file processed


In [48]:
final_df = pd.concat(result, ignore_index=True)

In [1]:
#final_df

## Validation

In [ ]:
print(f"There were {file_count} .json file(s) loaded into the folder\n")

print("Those files are: ")
for json_file in pathlib.Path(myPath).glob('*.json'):
    print(json_file.name)

loaded_file_count = len(pd.unique(final_df['file']))
print("")
print(f"There are {loaded_file_count} file(s) loaded into the DataFrame")
print("\nThose files are: ")
for item in pd.unique(final_df['file']):
    print(item)

Convert the above validation step to a code line and execute to view the number of json files stored in the file path provided, with the file names listed as well as the number of unique files stored in the DataFrame and their values. 